In [37]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset
import pandas as pd
from datetime import date, datetime, timedelta
import numpy as np
from azure.storage.blob import BlobServiceClient

subscription_id = '96599b9b-0d9b-4577-9163-418d2c6cd411'
resource_group = 'rg-datasci-ml-dev-001'
workspace_name = 'ml-moss-dev-001'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset_hourly_compare_cmic = Dataset.get_by_name(workspace, name='Hourly_Employee_Time_prod')
cmic_pr_df_dev = dataset_hourly_compare_cmic.to_pandas_dataframe()

dataset_hourly_compare_wd = Dataset.get_by_name(workspace, name='WD_Hourly_Employee_Time_prod')
wd_pr_df_dev = dataset_hourly_compare_wd.to_pandas_dataframe()

tcp_dataset = Dataset.get_by_name(workspace, name='WorkSegments_Timesheet_prod')
tcp_df_dev = tcp_dataset.to_pandas_dataframe()

jcjob_dataset = Dataset.get_by_name(workspace, name='JCJOB_TABLE_TN_prod')
jcjob_df = jcjob_dataset.to_pandas_dataframe()

In [38]:
###Step 1
#Filter Data Sets
#Data Manipulate Data Sets
#Narrow down to necessary columns only

In [39]:
##########################

week_ending = '2023-10-14'

##########################

date = datetime.strptime(week_ending, "%Y-%m-%d")
start_date = date - timedelta(days = 6)
end_date = date

In [40]:
tcp_df = tcp_df_dev.copy()

In [41]:
###TCP Data Manipulation
#Filter: Solar Jobs only 
tcp_df = tcp_df.loc[tcp_df['Job_Code'].str.startswith('710')]
#Filter: Only Keeping Labor Hourly
tcp_df = tcp_df.loc[tcp_df['Task'].isin(['LH','LHA'])]
#Creating a total hours column
tcp_df['total_hours'] = tcp_df['Regular_Hours'] + tcp_df['Overtime_Hours']
#Renaming Relevant Columns
tcp_df = tcp_df.rename({'Employee_Number':'EEID','Job_Code':'Job','Activity':'Sub_Job','total_hours':'Hours'},axis='columns')
tcp_cols_keep = ['EEID' , 'Date' , 'Job' , 'Sub_Job' , 'Hours']
tcp_df_comp = tcp_df.loc[:,tcp_cols_keep]

In [42]:
wd_pr_df = wd_pr_df_dev.copy()

In [43]:
#Workday Data Manipulation
#Drop Null values if the employee does not have a job/sub job assignment
wd_pr_df = wd_pr_df.loc[~wd_pr_df['cf_INTLRVCalculatedProjectFromTimeBlock.descriptor'].isna()]
#Filter: Solar Jobs Only
wd_pr_df = wd_pr_df.loc[wd_pr_df['cf_INTLRVCalculatedProjectFromTimeBlock.descriptor'].str.startswith('710')]
#This isolates the 2 digit sub job, also accomodates the sub jobs that have a letter in them like Claims (C) and Warranty (W)
wd_pr_df['cf_INTLRVCalculatedProjectFromTimeBlock.descriptor'] = wd_pr_df['cf_INTLRVCalculatedProjectFromTimeBlock.descriptor'].str.replace('C' , '.C')
wd_pr_df['cf_INTLRVCalculatedProjectFromTimeBlock.descriptor'] = wd_pr_df['cf_INTLRVCalculatedProjectFromTimeBlock.descriptor'].str.replace('W' , '.W')
wd_pr_df['sub_job'] = wd_pr_df['cf_INTLRVCalculatedProjectFromTimeBlock.descriptor'].str.split('.').str[1]
wd_pr_df['sub_job'] = wd_pr_df['sub_job'].str.split(' ').str[0]
#Leaves behind Job ONLY
wd_pr_df['job'] = wd_pr_df.loc[:,'cf_INTLRVCalculatedProjectFromTimeBlock.descriptor'].str[0:7]
#Renaming Relevant Columns
wd_pr_df = wd_pr_df.rename({'EmployeeID':'EEID','calculatedDate':'Date','job':'Job','sub_job':'Sub_Job','amount':'Hours'},axis='columns')
wd_cols_keep = ['EEID' , 'Date' , 'Job' , 'Sub_Job' , 'Hours']
wd_pr_df_comp = wd_pr_df.loc[:,wd_cols_keep]

In [59]:
wd_pr_df_dev.head(1)

,workdayID,EmployeeID,Employee_FullName,Workers,Employee_BillTransaction_Descriptor,billableTransaction.descriptor,Employee_BillTransactionDate,amount,dayOfTheWeek,FrequencySalary,...,timeType1.id,totalHoursTimeTracking,transactionDate2,transactionSource.descriptor,transactionSource.id,unit.descriptor,unit.id,unit1,unitOfTimeIsHours,unroundedDuration
0,1abc577aeaea9000c204fbb36b590000,10569,WILLIAM BROWN,WILLIAM BROWN (10569),10 Hours,10 Hours on 10/12/2023,10/12/2023,10.0,Thursday,Weekly Hourly,...,88572bc8f2d81000d2da8beacaec0000,10.0,2023-10-12,Time,7a55a775b63b4b9b8a5b3722fc1e4556,Hours,c4dacbd56bca4a9a8950e8d3ed21bbdb,Hours,True,0.0


In [62]:
#QC Work for Leanne on 10/16/2023 - Teams Chat
### EEID == '12713' showing zero in WD, 61 in TCP
#wd_pr_df_dev.loc[(wd_pr_df_dev['EmployeeID'] == '12713')]
wd_pr_df_comp.loc[wd_filter&(wd_pr_df_comp['EEID'] == '12713')]

,EEID,Date,Job,Sub_Job,Hours


In [52]:
tcp_df_comp.loc[tcp_filter&(tcp_df_comp['EEID'] == '12713')]

,EEID,Date,Job,Sub_Job,Hours
184421,12713,2023-10-09,7102120,25,10.5
188984,12713,2023-10-08,7102120,25,8.5
188988,12713,2023-10-10,7102120,25,10.5
189007,12713,2023-10-13,7102120,25,10.5
216072,12713,2023-10-12,7102120,25,10.5
217634,12713,2023-10-11,7102120,25,10.5


In [48]:
#QC Work for Leanne on 10/16/2023 - Teams Chat
### ['EEID'] == '98050' showing 52 in TCP, 
wd_pr_df_comp.loc[wd_pr_df_comp['EEID'] == '98050']

,EEID,Date,Job,Sub_Job,Hours


In [47]:
#QC Work for Leanne on 10/16/2023 - Teams Chat
### ['EEID'] == '98050' showing 52 in TCP, 
tcp_df_comp.loc[(tcp_filter)&(tcp_df_comp['EEID']=='98050')]

,EEID,Date,Job,Sub_Job,Hours
727467,98050,2023-10-09,7102115,10,10.0
727473,98050,2023-10-14,7102115,10,2.0
812963,98050,2023-10-12,7102115,10,10.0
818942,98050,2023-10-10,7102115,10,10.0
818943,98050,2023-10-13,7102115,10,10.0
828880,98050,2023-10-11,7102115,10,10.0


In [59]:
cmic_pr_df = cmic_pr_df_dev.copy()

In [60]:
#CMiC Data Manipulation
#Re-casting this column as a Date data type
cmic_pr_df["Date"] = pd.to_datetime(cmic_pr_df["Date"])
#Filter: Only Keeping Labor Hourly
cmic_pr_df = cmic_pr_df.loc[cmic_pr_df['Task'].isin(['LH','LHA'])]
#Renaming Relevant Columns
cmic_pr_df = cmic_pr_df.rename({'Employee Number':'EEID','Job Code':'Job','Sub Job Code':'Sub_Job','Regular Hours':'Hours'},axis='columns')
cmic_cols_keep = ['EEID' , 'Date' , 'Job' , 'Sub_Job' , 'Hours']
cmic_pr_df_comp = cmic_pr_df.loc[:,cmic_cols_keep]

In [61]:
###Step 2
#Create filter that works for all 3 data sets
#Group by data 
#join data for comparison (3 joins)(suffixes on columns)(Outer Join)
#Missing EEIDs

###Step 3
#Add Job Names
#Job Total Comparisons
#Job and Sub Job Total Comparisons
#Create separate DFs for missing Jobs/EEIDs paired with subjob - TBD

In [107]:
#Date Filters
tcp_filter = (tcp_df_comp['Date']>=start_date)&(tcp_df_comp['Date']<=end_date)
cmic_filter = (cmic_pr_df_comp['Date']>=start_date)&(cmic_pr_df_comp['Date']<=end_date)
wd_filter = (wd_pr_df_comp['Date']>=start_date)&(wd_pr_df_comp['Date']<=end_date)

# FIGURE OUT CMIC DATE FILTER - MAX DATE ISSUE

In [63]:
#cmic_pr_df_comp.loc[cmic_pr_df_comp['Date']>='2023-09-23']
#cmic_pr_df_comp.loc[cmic_filter]
#wd_pr_df_comp.loc[wd_pr_df_comp['Date']>='2023-10-01']
#wd_pr_df_comp.loc[wd_filter]
#tcp_df_comp.loc[tcp_filter]
cmic_pr_df_comp.Date.max()

Timestamp('2023-09-29 00:00:00')

#### Grouping the Data by Different Levels

In [64]:
#Grouping by Job
tcp_grouped_job = tcp_df_comp.loc[tcp_filter].groupby(['Job']).agg(TotalHours_TCP = pd.NamedAgg(column='Hours',aggfunc='sum'))
tcp_grouped_job = tcp_grouped_job.reset_index()
cmic_grouped_job = cmic_pr_df_comp.loc[cmic_filter].groupby(['Job']).agg(TotalHours_CMiC = pd.NamedAgg(column='Hours',aggfunc='sum'))
cmic_grouped_job = cmic_grouped_job.reset_index()
wd_grouped_job = wd_pr_df_comp.loc[wd_filter].groupby(['Job']).agg(TotalHours_WD = pd.NamedAgg(column='Hours',aggfunc='sum'))
wd_grouped_job = wd_grouped_job.reset_index()

In [65]:
#Grouping by Job, Sub Job
tcp_grouped_subjob = tcp_df_comp.loc[tcp_filter].groupby(['Job','Sub_Job']).agg(TotalHours_TCP = pd.NamedAgg(column='Hours',aggfunc='sum'))
tcp_grouped_subjob = tcp_grouped_subjob.reset_index()
cmic_grouped_subjob = cmic_pr_df_comp.loc[cmic_filter].groupby(['Job','Sub_Job']).agg(TotalHours_CMiC = pd.NamedAgg(column='Hours',aggfunc='sum'))
cmic_grouped_subjob = cmic_grouped_subjob.reset_index()
wd_grouped_subjob = wd_pr_df_comp.loc[wd_filter].groupby(['Job','Sub_Job']).agg(TotalHours_WD = pd.NamedAgg(column='Hours',aggfunc='sum'))
wd_grouped_subjob = wd_grouped_subjob.reset_index()

In [66]:
#Grouping by Job, Sub Job, EEID
### Commented out for EEID Comparison Only - uncomment out to compare EEID by Job and Sub Job
#tcp_grouped_ee = tcp_df_comp.loc[tcp_filter].groupby(['Job','Sub_Job','EEID']).agg(TotalHours_TCP = pd.NamedAgg(column='Hours',aggfunc='sum'))
tcp_grouped_ee = tcp_df_comp.loc[tcp_filter].groupby(['EEID']).agg(TotalHours_TCP = pd.NamedAgg(column='Hours',aggfunc='sum'))
tcp_grouped_ee = tcp_grouped_ee.reset_index()
### Commented out for EEID Comparison Only - uncomment out to compare EEID by Job and Sub Job
#cmic_grouped_ee = cmic_pr_df_comp.loc[cmic_filter].groupby(['Job','Sub_Job','EEID']).agg(TotalHours_CMiC = pd.NamedAgg(column='Hours',aggfunc='sum'))
cmic_grouped_ee = cmic_pr_df_comp.loc[cmic_filter].groupby(['EEID']).agg(TotalHours_CMiC = pd.NamedAgg(column='Hours',aggfunc='sum'))
cmic_grouped_ee = cmic_grouped_ee.reset_index()
### Commented out for EEID Comparison Only - uncomment out to compare EEID by Job and Sub Job
#wd_grouped_ee = wd_pr_df_comp.loc[wd_filter].groupby(['Job','Sub_Job','EEID']).agg(TotalHours_WD = pd.NamedAgg(column='Hours',aggfunc='sum'))
wd_grouped_ee = wd_pr_df_comp.loc[wd_filter].groupby(['EEID']).agg(TotalHours_WD = pd.NamedAgg(column='Hours',aggfunc='sum'))
wd_grouped_ee = wd_grouped_ee.reset_index()

#### Unique Column Creation for the Joins

In [67]:
#Unique Columns to Join On for Sub Job Comparison
tcp_grouped_subjob['unique'] = tcp_grouped_subjob[['Job','Sub_Job']].apply(lambda x: ' - '.join(str(value) for value in x), axis=1)
cmic_grouped_subjob['unique'] = cmic_grouped_subjob[['Job','Sub_Job']].apply(lambda x: ' - '.join(str(value) for value in x), axis=1)
wd_grouped_subjob['unique'] = wd_grouped_subjob[['Job','Sub_Job']].apply(lambda x: ' - '.join(str(value) for value in x), axis=1)

In [68]:
#Unique Columns to Join On for EEID Comparison
### Commented out for EEID Comparison Only - uncomment out to compare EEID by Job and Sub Job
#tcp_grouped_ee['unique'] = tcp_grouped_ee[['EEID','Job','Sub_Job']].apply(lambda x: ' - '.join(str(value) for value in x), axis=1)
#cmic_grouped_ee['unique'] = cmic_grouped_ee[['EEID','Job','Sub_Job']].apply(lambda x: ' - '.join(str(value) for value in x), axis=1)
#wd_grouped_ee['unique'] = wd_grouped_ee[['EEID','Job','Sub_Job']].apply(lambda x: ' - '.join(str(value) for value in x), axis=1)

#### TCP vs. Workday Comparison

In [70]:
#TCP v. WD Job
tcp_v_wd_job = tcp_grouped_job.merge(wd_grouped_job , how = 'outer' , on = 'Job' , suffixes = ['_TCP','_WD'])
#NaN Hours will be filled with 0
tcp_v_wd_job['TotalHours_WD'] = tcp_v_wd_job['TotalHours_WD'].fillna(0)
tcp_v_wd_job['TotalHours_TCP'] = tcp_v_wd_job['TotalHours_TCP'].fillna(0)
tcp_v_wd_job['Hours_Variance (TCP - WD)'] = tcp_v_wd_job['TotalHours_TCP'] - tcp_v_wd_job['TotalHours_WD']
tcp_v_wd_job = tcp_v_wd_job.sort_values(['Job'  , 'Hours_Variance (TCP - WD)'] , ascending=[False,False])

In [71]:
#TCP v. WD Job, Sub Job
tcp_v_wd_subjob = tcp_grouped_subjob.merge(wd_grouped_subjob , how = 'outer' , on = 'unique' , suffixes = ['_TCP','_WD']).drop(columns =['unique'])
#NaN Hours will be filled with 0
tcp_v_wd_subjob['TotalHours_WD'] = tcp_v_wd_subjob['TotalHours_WD'].fillna(0)
tcp_v_wd_subjob['TotalHours_TCP'] = tcp_v_wd_subjob['TotalHours_TCP'].fillna(0)
tcp_v_wd_subjob['Hours_Variance (TCP - WD)'] = tcp_v_wd_subjob['TotalHours_TCP'] - tcp_v_wd_subjob['TotalHours_WD']
tcp_v_wd_subjob = tcp_v_wd_subjob.sort_values(['Job_TCP' , 'Sub_Job_TCP' , 'Hours_Variance (TCP - WD)'] , ascending=[False,True,False])

In [72]:
#TCP v. WD Job, Sub Job, EEID
### Commented out for EEID Comparison Only - uncomment out to compare EEID by Job and Sub Job
#tcp_v_wd_ee = tcp_grouped_ee.merge(wd_grouped_ee , how = 'outer' , on = 'unique' , suffixes = ['_TCP','_WD']).drop(columns =['unique'])
tcp_v_wd_ee = tcp_grouped_ee.merge(wd_grouped_ee , how = 'outer' , on = 'EEID' , suffixes = ['_TCP','_WD'])
#NaN Hours will be filled with 0
tcp_v_wd_ee['TotalHours_WD'] = tcp_v_wd_ee['TotalHours_WD'].fillna(0)
tcp_v_wd_ee['TotalHours_TCP'] = tcp_v_wd_ee['TotalHours_TCP'].fillna(0)
tcp_v_wd_ee['Hours_Variance (TCP - WD)'] = tcp_v_wd_ee['TotalHours_TCP'] - tcp_v_wd_ee['TotalHours_WD']
tcp_v_wd_ee = tcp_v_wd_ee.sort_values('Hours_Variance (TCP - WD)' , ascending=False, key=abs)
#Wait for direction from stakeholder

#Filter for 0s OR Nulls in columns and create new DF: Missing_In_WD_df, Missing_In_TCP_df 
#Filter down to relevant columns only
#tcp_v_wd_ee = tcp_v_wd_ee.loc[:,['Job_TCP','Sub_Job_TCP','EEID_TCP','TotalHours_TCP','TotalHours_WD','Hours_Variance (TCP - WD)']]
#Fill NaNs with Text: "Job Missing in Workday" or "EEID Missing in Workday"

#### TCP vs. CMiC Comparison

In [73]:
#TCP v CMiC Job
tcp_v_cmic_job = tcp_grouped_job.merge(cmic_grouped_job , how = 'outer' , on = 'Job' , suffixes = ['','_CMiC'])
#NaN Hours will be filled with 0
tcp_v_cmic_job['TotalHours_CMiC'] = tcp_v_cmic_job['TotalHours_CMiC'].fillna(0)
tcp_v_cmic_job['TotalHours_TCP'] = tcp_v_cmic_job['TotalHours_TCP'].fillna(0)
tcp_v_cmic_job['Hours_Variance (TCP - CMiC)'] = tcp_v_cmic_job['TotalHours_TCP'] - tcp_v_cmic_job['TotalHours_CMiC']
tcp_v_cmic_job = tcp_v_cmic_job.sort_values(['Job' , 'Hours_Variance (TCP - CMiC)'], ascending=[False, False])

In [74]:
#TCP v CMiC Job, Sub Job
tcp_v_cmic_subjob = tcp_grouped_subjob.merge(cmic_grouped_subjob , how = 'outer' , on = 'unique' , suffixes = ['','_CMiC']).drop(columns =['unique'])
#NaN Hours will be filled with 0
tcp_v_cmic_subjob['TotalHours_CMiC'] = tcp_v_cmic_subjob['TotalHours_CMiC'].fillna(0)
tcp_v_cmic_subjob['TotalHours_TCP'] = tcp_v_cmic_subjob['TotalHours_TCP'].fillna(0)
tcp_v_cmic_subjob['Hours_Variance (TCP - CMiC)'] = tcp_v_cmic_subjob['TotalHours_TCP'] - tcp_v_cmic_subjob['TotalHours_CMiC']
tcp_v_cmic_subjob = tcp_v_cmic_subjob.sort_values(['Job' , 'Sub_Job', 'Hours_Variance (TCP - CMiC)'], ascending=[False, True, False])

In [75]:
#TCP v CMiC Job, Sub Job, EEID
### Commented out for EEID Comparison Only - uncomment out to compare EEID by Job and Sub Job
#tcp_v_cmic_ee = tcp_grouped_ee.merge(cmic_grouped_ee , how = 'outer' , on = 'unique' , suffixes = ['','_CMiC']).drop(columns =['unique'])
tcp_v_cmic_ee = tcp_grouped_ee.merge(cmic_grouped_ee , how = 'outer' , on = 'EEID' , suffixes = ['','_CMiC'])
#NaN Hours will be filled with 0
tcp_v_cmic_ee['TotalHours_CMiC'] = tcp_v_cmic_ee['TotalHours_CMiC'].fillna(0)
tcp_v_cmic_ee['TotalHours_TCP'] = tcp_v_cmic_ee['TotalHours_TCP'].fillna(0)
tcp_v_cmic_ee['Hours_Variance (TCP - CMiC)'] = tcp_v_cmic_ee['TotalHours_TCP'] - tcp_v_cmic_ee['TotalHours_CMiC']
tcp_v_cmic_ee = tcp_v_cmic_ee.sort_values('Hours_Variance (TCP - CMiC)', ascending=False, key=abs)
#Filter down to relevant columns only

#### Workday vs. CMiC Comparison

In [77]:
#WD v. CMiC EEID
wd_v_cmic_job = wd_grouped_job.merge(cmic_grouped_job , how = 'outer' , on = 'Job' , suffixes = ['_WD','_CMiC'])
#NaN Hours will be filled with 0
wd_v_cmic_job['TotalHours_CMiC'] = wd_v_cmic_job['TotalHours_CMiC'].fillna(0)
wd_v_cmic_job['TotalHours_WD'] = wd_v_cmic_job['TotalHours_WD'].fillna(0)
wd_v_cmic_job['Hours_Variance (WD - CMiC)'] = wd_v_cmic_job['TotalHours_WD'] - wd_v_cmic_job['TotalHours_CMiC']
wd_v_cmic_job = wd_v_cmic_job.sort_values(['Job' , 'Hours_Variance (WD - CMiC)'], ascending=[False,False])

In [78]:
#WD v. CMiC Job, Sub Job
wd_v_cmic_subjob = wd_grouped_subjob.merge(cmic_grouped_subjob , how = 'outer' , on = 'unique' , suffixes = ['_WD','_CMiC']).drop(columns =['unique'])
#NaN Hours will be filled with 0
wd_v_cmic_subjob['TotalHours_CMiC'] = wd_v_cmic_subjob['TotalHours_CMiC'].fillna(0)
wd_v_cmic_subjob['TotalHours_WD'] = wd_v_cmic_subjob['TotalHours_WD'].fillna(0)
wd_v_cmic_subjob['Hours_Variance (WD - CMiC)'] = wd_v_cmic_subjob['TotalHours_WD'] - wd_v_cmic_subjob['TotalHours_CMiC']
wd_v_cmic_subjob = wd_v_cmic_subjob.sort_values(['Job_WD' , 'Sub_Job_WD' , 'Hours_Variance (WD - CMiC)'], ascending=[False,True,False])

In [79]:
#WD v. CMiC Job, Sub Job, EEID
### Commented out for EEID Comparison Only - uncomment out to compare EEID by Job and Sub Job
#wd_v_cmic_ee = wd_grouped_ee.merge(cmic_grouped_ee , how = 'outer' , on = 'unique' , suffixes = ['_WD','_CMiC']).drop(columns =['unique'])
wd_v_cmic_ee = wd_grouped_ee.merge(cmic_grouped_ee , how = 'outer' , on = 'EEID' , suffixes = ['_WD','_CMiC'])
#NaN Hours will be filled with 0
wd_v_cmic_ee['TotalHours_CMiC'] = wd_v_cmic_ee['TotalHours_CMiC'].fillna(0)
wd_v_cmic_ee['TotalHours_WD'] = wd_v_cmic_ee['TotalHours_WD'].fillna(0)
wd_v_cmic_ee['Hours_Variance (WD - CMiC)'] = wd_v_cmic_ee['TotalHours_WD'] - wd_v_cmic_ee['TotalHours_CMiC']
wd_v_cmic_ee = wd_v_cmic_ee.sort_values('Hours_Variance (WD - CMiC)', ascending=False , key=abs)
#Filter down to relevant columns only

In [80]:
file_path = os.getcwd() + '/excel_output'
file_path

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/jbeachy2/code/Users/JBeachy/Project_DataDomain/Payroll/DataDelivery_Compare/Hours_Comparison/excel_output'

In [81]:
#File Path in AMLS
file_name = f'Hours_Comparison_WeekEnding_{week_ending}'
#Writing to AMLS File Directory
with pd.ExcelWriter(f'{file_path}/{file_name}.xlsx') as writer:
    tcp_v_wd_job.to_excel(writer , sheet_name = 'TCPvWD_Hours_byJob' , index = False)
    tcp_v_wd_subjob.to_excel(writer , sheet_name = 'TCPvWD_Hours_bySubJob' , index = False)
    tcp_v_wd_ee.to_excel(writer , sheet_name = 'TCPvWD_Hours_byEE' , index = False)
    tcp_v_cmic_job.to_excel(writer , sheet_name = 'TCPvCMiC_Hours_byJob' , index = False)
    tcp_v_cmic_subjob.to_excel(writer , sheet_name = 'TCPvCMiC_Hours_bySubJob' , index = False)
    tcp_v_cmic_ee.to_excel(writer , sheet_name = 'TCPvCMiC_Hours_byEE' , index = False)
    wd_v_cmic_job.to_excel(writer , sheet_name = 'WDvCMiC_Hours_byJob' , index = False)
    wd_v_cmic_subjob.to_excel(writer , sheet_name = 'WDvCMiC_Hours_bySubJob' , index = False)
    wd_v_cmic_ee.to_excel(writer , sheet_name = 'WDvCMiC_Hours_byEE' , index = False)

In [82]:
#Blob Storage Info
storage_account_key = 'u46QXBaayH/rWljcqPTWZTNgFdGHo9zH4I0OLWgDi4oa2inkpjVwTbp74C+ISDC2oWtNlrcr69Ec+ASt5wV1PA=='
storage_account_name = 'mossdatalakesource'
connection_string = 'DefaultEndpointsProtocol=https;AccountName=mossdatalakesource;AccountKey=u46QXBaayH/rWljcqPTWZTNgFdGHo9zH4I0OLWgDi4oa2inkpjVwTbp74C+ISDC2oWtNlrcr69Ec+ASt5wV1PA==;EndpointSuffix=core.windows.net'
container_name = 'cmic'
#Defininng Blob Storage Uplpoad#
def uploadtoblobstorage(file_path,file_name):
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)
    blob_client = blob_service_client.get_blob_client(container = container_name , blob = file_name)

    with open(file_path , 'rb') as data:
        blob_client.upload_blob(data)
    print(f'Uploaded {file_name}.')

In [83]:
file_name_exp = f'{file_name}.xlsx'
uploadtoblobstorage(f'{file_path}/{file_name}.xlsx', file_name_exp)

Uploaded Hours_Comparison_WeekEnding_2023-10-14.xlsx.
